In [1]:
import pandas as pd
import numpy as np
import torch
from gensim.models import Word2Vec
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D, Concatenate, Input, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

2024-10-29 02:27:33.245224: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-29 02:27:35.221938: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-10-29 02:27:35.221974: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-10-29 02:27:35.448607: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-29 02:27:38.461406: W tensorflow/stream_executor/platform/de

In [2]:
#chessData = pd.read_csv('Data/chessData.csv')
#random_evals = pd.read_csv('Data/random_evals.csv')
tactic_evals = pd.read_csv('Data/tactic_evals.csv')

In [3]:
len(tactic_evals)
tactic_evals = tactic_evals[:500000]


In [4]:
tactic_evals_check = tactic_evals.dropna()
tactic_evals_check['isCheck'] = tactic_evals_check['Evaluation'].str.contains("#")
tactic_evals_check['isCheck'] = list(map(int, tactic_evals_check['isCheck']))

predictors_raw = tactic_evals_check.drop(['Evaluation', 'isCheck'], axis=1)
predictors_raw.head()

/tmp/ipykernel_4538/2706941485.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tactic_evals_check['isCheck'] = tactic_evals_check['Evaluation'].str.contains("#")
/tmp/ipykernel_4538/2706941485.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tactic_evals_check['isCheck'] = list(map(int, tactic_evals_check['isCheck']))


,FEN,Move
0,r2qkbr1/pb1nn3/1ppp3p/8/3P1p2/2PB1N1P/PPQN1PP1...,d3g6
1,r2qkb2/pb1nn3/1ppp2rp/8/3P1p2/2P2N1P/PPQN1PP1/...,c2g6
2,r2qkbr1/pb1nn3/1ppp2Bp/8/3P1p2/2P2N1P/PPQN1PP1...,g8g6
3,8/7p/R5p1/2p1pkP1/7P/P4PK1/1r6/3q4 w - - 6 46,a6f6
4,6k1/pp6/3p4/2p1p3/2P1P1q1/1P1P2pP/P5P1/5K2 w -...,h3g4


In [5]:
def separate_board(board):
    return board.replace('/', ' ').split()

def separate_move(move):
    separated_move = []
    for i in range(len(move)):
        separated_move.append(move[i])
    return separated_move


In [6]:
tokens_lines = list(map(separate_board, np.asarray(tactic_evals_check['FEN'])))
tokens_lines = [line[0:8] for line in tokens_lines]

tokens_moves = list(map(separate_move, np.asarray(tactic_evals_check['Move'])))
#tokens_piece = list(map(fen_to_tokens_without_spaces, np.asarray(tactic_evals['FEN'])))
predictors_raw['FEN'] = tokens_lines
predictors_raw['Move'] = tokens_moves


In [7]:
fen_embedding_dim = 64
fen_embeddings = Word2Vec(sentences=tokens_lines, vector_size=fen_embedding_dim, window=5, min_count=1, sg=1)
fen_embeddings.train(tokens_lines, total_examples=len(tokens_lines), epochs=10)

(31168484, 37201280)

In [8]:
move_embedding_dim = 64
move_embeddings = Word2Vec(sentences=tokens_moves, vector_size=move_embedding_dim, window=5, min_count=1, sg=1)
move_embeddings.train(tokens_moves, total_examples=len(tokens_moves), epochs=10)

(2647889, 18619900)

In [9]:
fen_data = []
move_data = []

for line in range(len(tokens_lines)):
    fen_data.append(fen_embeddings.wv[tokens_lines[line]])

for move in range(len(tokens_moves)):
    move_data.append(move_embeddings.wv[tokens_moves[move]])

predictors = pd.DataFrame({
    'fen_data': fen_data,
    'move_data': move_data
})

In [10]:
move_data_expanded = []
for embedding in move_data:
    if embedding.shape == (4, 64):
        move_data_expanded.append(np.vstack([embedding, np.zeros((4, 64))]))
    else:
        move_data_expanded.append(np.vstack([embedding, np.zeros((3, 64))]))
        
predictors['move_data'] = move_data_expanded

In [11]:
print(f"fen_data_shape: {predictors['fen_data'][0].shape}\n move_data_shape: {predictors['move_data'][0].shape}")


fen_data_shape: (8, 64)
 move_data_shape: (8, 64)


In [12]:
predictors.head()

,fen_data,move_data
0,"[[0.11849159, -0.08676035, 0.19801097, 0.15948...","[[-0.026441823691129684, 0.11011077463626862, ..."
1,"[[0.13075054, 0.057045773, 0.1320473, -0.08782...","[[0.14347383379936218, 0.22420170903205872, 0...."
2,"[[0.11849159, -0.08676035, 0.19801097, 0.15948...","[[0.18018083274364471, 0.10428038984537125, 0...."
3,"[[0.24744636, 0.11220646, 0.36438856, 0.325321...","[[0.057891324162483215, 0.4406301975250244, 0...."
4,"[[-0.1427337, 0.18077302, 0.49280927, -0.20337...","[[0.1296084225177765, 0.3589269816875458, 0.26..."


In [13]:
ncols = predictors.shape[1]

fen_input = Input(shape=(8, 64, 1), name='fen_input')
move_input = Input(shape=(8, 64, 1), name='move_input')

fen_conv1 = Conv2D(32, (2,2), activation='relu')(fen_input)
move_conv1 = Conv2D(32, (2,2), activation='relu')(move_input)

fen_mp1 = MaxPooling2D(pool_size=(2,2))(fen_conv1)
move_mp1 = MaxPooling2D(pool_size=(2,2))(move_conv1)

fen_flatter = Flatten()(fen_mp1)
move_flatter = Flatten()(move_mp1)

combined = Concatenate()([fen_flatter, move_flatter])

dense = (Dense(64, activation = 'relu'))(combined)
output = (Dense(1 , activation = 'sigmoid'))(dense)

mate_evaluator = Model(inputs=[fen_input, move_input], outputs=output)
mate_evaluator.compile(optimizer = 'adam', loss = 'categorical_crossentropy', 
		      metrics=['accuracy'])

mate_evaluator.summary()

2024-10-29 02:29:40.955943: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-10-29 02:29:40.955965: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2024-10-29 02:29:40.955989: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (gabs-IdeaPad-3-15ALC6): /proc/driver/nvidia/version does not exist
2024-10-29 02:29:40.959865: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 fen_input (InputLayer)         [(None, 8, 64, 1)]   0           []                               
                                                                                                  
 move_input (InputLayer)        [(None, 8, 64, 1)]   0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 7, 63, 32)    160         ['fen_input[0][0]']              
                                                                                                  
 conv2d_1 (Conv2D)              (None, 7, 63, 32)    160         ['move_input[0][0]']             
                                                                                              

In [14]:
print(predictors['move_data'][0].shape)
for move in predictors['move_data']:
    if move.shape != (8, 64):
        print(move.shape)

(8, 64)


In [15]:
# Extraindo as colunas do DataFrame predictors
fen_data = np.array(predictors['fen_data'].tolist())  # ou predictors['fen_data'].values
move_data = np.array(predictors['move_data'].tolist())  # ou predictors['move_data'].values

# Verifique a forma dos dados extraídos
print(fen_data.shape)  # Espera-se que seja (465016, 8, 64)
print(move_data.shape)  # Espera-se que seja (465016, 8, 64)

# Empilhe os dados para obter a forma desejada
X_fen = np.reshape(fen_data, (fen_data.shape[0], 8, 64, 1))  # Reestrutura para (n_samples, 8, 64, 1)
X_move = np.reshape(move_data, (move_data.shape[0], 8, 64, 1))  # Reestrutura para (n_samples, 8, 64, 1)

# Prepare os rótulos
y = tactic_evals_check['isCheck'].to_numpy()  # ou tactic_evals_check['isCheck'].values

# Verifique a forma dos dados
print(X_fen.shape)  # Deve ser (465016, 8, 64, 1)
print(X_move.shape)  # Deve ser (465016, 8, 64, 1)
print(y.shape)  # Deve ser (465016,)


(465016, 8, 64)
(465016, 8, 64)
(465016, 8, 64, 1)
(465016, 8, 64, 1)
(465016,)


In [ ]:
early_stopping_monitor = EarlyStopping(patience=2)
mate_evaluator.fit([X_fen, X_move], y, 
                    validation_split=0.3, 
                    epochs=20, 
                    callbacks=[early_stopping_monitor])